In [1]:
# основной модль выполнения запросов spark italist
# 4 этапа запроса с помощью spark italist
1) анализ когда вычисляются  отношения из абстрактного синтаксического дерева возвращаемого синтаксическим анализатором sql либо из обьекта df созданного api 

In [ ]:
логическая оптимизация типовые правила применяются к логическому плану свертак придекатное сжатие сокращение 
физическая оптимизация один или несколько физических планов формируются из логического с ипользованием физического оператора 

In [4]:
import pyspark

from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName('less_pysqprk_DF').\
        master('local').\
        config('spark.executor.memory', "512m").\
        config("spark.hadoop.fs.defaultFS", "hdfs://localhost:9000").\
        config("spark.hadoop.fs.default.name", "hdfs://localhost:9000").\
        getOrCreate()

/home/ilyagasan/anaconda3/lib/python3.7/site-packages/pyspark/context.py:317: FutureWarning: Python 3.7 support is deprecated in Spark 3.4.
  warnings.warn("Python 3.7 support is deprecated in Spark 3.4.", FutureWarning)


In [2]:
import os

# Установите переменную окружения JAVA_HOME
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'
os.environ['PATH'] = os.environ['JAVA_HOME'] + '/bin:' + os.environ['PATH']

# Проверьте версию Java
!java -version

openjdk version "1.8.0_422"
OpenJDK Runtime Environment (build 1.8.0_422-8u422-b05-1~24.04-b05)
OpenJDK 64-Bit Server VM (build 25.422-b05, mixed mode)


In [6]:
df = spark.read.csv('hdfs://localhost:9000/data/Sales_April_2019.csv', header=True, inferSchema=True)

In [15]:
df.show()

+--------+--------------------+----------------+----------+--------------+--------------------+
|Order ID|             Product|Quantity Ordered|Price Each|    Order Date|    Purchase Address|
+--------+--------------------+----------------+----------+--------------+--------------------+
|  176558|USB-C Charging Cable|               2|     11.95|04/19/19 08:46|917 1st St, Dalla...|
|    null|                null|            null|      null|          null|                null|
|  176559|Bose SoundSport H...|               1|     99.99|04/07/19 22:30|682 Chestnut St, ...|
|  176560|        Google Phone|               1|     600.0|04/12/19 14:38|669 Spruce St, Lo...|
|  176560|    Wired Headphones|               1|     11.99|04/12/19 14:38|669 Spruce St, Lo...|
|  176561|    Wired Headphones|               1|     11.99|04/30/19 09:27|333 8th St, Los A...|
|  176562|USB-C Charging Cable|               1|     11.95|04/29/19 13:03|381 Wilson St, Sa...|
|  176563|Bose SoundSport H...|         

In [8]:
df.explain(True)

== Parsed Logical Plan ==
Relation [Order ID#17,Product#18,Quantity Ordered#19,Price Each#20,Order Date#21,Purchase Address#22] csv

== Analyzed Logical Plan ==
Order ID: int, Product: string, Quantity Ordered: int, Price Each: double, Order Date: string, Purchase Address: string
Relation [Order ID#17,Product#18,Quantity Ordered#19,Price Each#20,Order Date#21,Purchase Address#22] csv

== Optimized Logical Plan ==
Relation [Order ID#17,Product#18,Quantity Ordered#19,Price Each#20,Order Date#21,Purchase Address#22] csv

== Physical Plan ==
FileScan csv [Order ID#17,Product#18,Quantity Ordered#19,Price Each#20,Order Date#21,Purchase Address#22] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[hdfs://localhost:9000/data/Sales_April_2019.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Order ID:int,Product:string,Quantity Ordered:int,Price Each:double,Order Date:string,Purch...



In [26]:
df.explain()


== Physical Plan ==
*(1) Project [Order ID#17, Product#18, coalesce(Quantity Ordered#19, 20) AS Quantity Ordered#98, Price Each#20, Order Date#21, Purchase Address#22]
+- FileScan csv [Order ID#17,Product#18,Quantity Ordered#19,Price Each#20,Order Date#21,Purchase Address#22] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[hdfs://localhost:9000/data/Sales_April_2019.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Order ID:int,Product:string,Quantity Ordered:int,Price Each:double,Order Date:string,Purch...




In [13]:
from pyspark.sql.functions import col
df.filter(col('Price Each') > 20).explain()

== Physical Plan ==
*(1) Filter (isnotnull(Price Each#20) AND (Price Each#20 > 20.0))
+- FileScan csv [Order ID#17,Product#18,Quantity Ordered#19,Price Each#20,Order Date#21,Purchase Address#22] Batched: false, DataFilters: [isnotnull(Price Each#20), (Price Each#20 > 20.0)], Format: CSV, Location: InMemoryFileIndex(1 paths)[hdfs://localhost:9000/data/Sales_April_2019.csv], PartitionFilters: [], PushedFilters: [IsNotNull(Price Each), GreaterThan(Price Each,20.0)], ReadSchema: struct<Order ID:int,Product:string,Quantity Ordered:int,Price Each:double,Order Date:string,Purch...




In [14]:
df.printSchema()

root
 |-- Order ID: integer (nullable = true)
 |-- Product: string (nullable = true)
 |-- Quantity Ordered: integer (nullable = true)
 |-- Price Each: double (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Purchase Address: string (nullable = true)



In [24]:
from pyspark.sql.functions import coalesce, lit
df = df.withColumn('Quantity Ordered', coalesce(col('Quantity Ordered'), lit(20)))

In [25]:
df.show()

+--------+--------------------+----------------+----------+--------------+--------------------+
|Order ID|             Product|Quantity Ordered|Price Each|    Order Date|    Purchase Address|
+--------+--------------------+----------------+----------+--------------+--------------------+
|  176558|USB-C Charging Cable|               2|     11.95|04/19/19 08:46|917 1st St, Dalla...|
|    null|                null|              20|      null|          null|                null|
|  176559|Bose SoundSport H...|               1|     99.99|04/07/19 22:30|682 Chestnut St, ...|
|  176560|        Google Phone|               1|     600.0|04/12/19 14:38|669 Spruce St, Lo...|
|  176560|    Wired Headphones|               1|     11.99|04/12/19 14:38|669 Spruce St, Lo...|
|  176561|    Wired Headphones|               1|     11.99|04/30/19 09:27|333 8th St, Los A...|
|  176562|USB-C Charging Cable|               1|     11.95|04/29/19 13:03|381 Wilson St, Sa...|
|  176563|Bose SoundSport H...|         

In [29]:
df.filter(col('Quantity Ordered') == 20).show()

+--------+-------+----------------+----------+----------+----------------+
|Order ID|Product|Quantity Ordered|Price Each|Order Date|Purchase Address|
+--------+-------+----------------+----------+----------+----------------+
|    null|   null|              20|      null|      null|            null|
|    null|Product|              20|      null|Order Date|Purchase Address|
+--------+-------+----------------+----------+----------+----------------+



In [27]:
df = df.drop_duplicates()

In [30]:
# запись в кеш 

In [31]:
df = df.cache()

In [32]:
df.show()

+--------+--------------------+----------------+----------+--------------+--------------------+
|Order ID|             Product|Quantity Ordered|Price Each|    Order Date|    Purchase Address|
+--------+--------------------+----------------+----------+--------------+--------------------+
|  176675|AAA Batteries (4-...|               1|      2.99|04/02/19 09:02|933 Meadow St, Se...|
|  176725|AA Batteries (4-p...|               2|      3.84|04/15/19 14:33|630 14th St, San ...|
|  176925|Lightning Chargin...|               1|     14.95|04/18/19 15:31|920 Wilson St, Ne...|
|  176991|Bose SoundSport H...|               1|     99.99|04/15/19 01:49|338 2nd St, San F...|
|  177141|AAA Batteries (4-...|               1|      2.99|04/20/19 16:12|559 4th St, San F...|
|  177177|AA Batteries (4-p...|               1|      3.84|04/08/19 12:10|182 12th St, San ...|
|  177221|34in Ultrawide Mo...|               1|    379.99|04/12/19 20:33|5 Hickory St, San...|
|  177239|              iPhone|         

In [33]:
df.explain()

== Physical Plan ==
InMemoryTableScan [Order ID#17, Product#18, Quantity Ordered#98, Price Each#20, Order Date#21, Purchase Address#22]
   +- InMemoryRelation [Order ID#17, Product#18, Quantity Ordered#98, Price Each#20, Order Date#21, Purchase Address#22], StorageLevel(disk, memory, deserialized, 1 replicas)
         +- *(2) HashAggregate(keys=[Order ID#17, Price Each#20, Quantity Ordered#98, Product#18, Order Date#21, Purchase Address#22], functions=[])
            +- Exchange hashpartitioning(Order ID#17, Price Each#20, Quantity Ordered#98, Product#18, Order Date#21, Purchase Address#22, 200), ENSURE_REQUIREMENTS, [plan_id=152]
               +- *(1) HashAggregate(keys=[Order ID#17, knownfloatingpointnormalized(normalizenanandzero(Price Each#20)) AS Price Each#20, Quantity Ordered#98, Product#18, Order Date#21, Purchase Address#22], functions=[])
                  +- *(1) Project [Order ID#17, Product#18, coalesce(Quantity Ordered#19, 20) AS Quantity Ordered#98, Price Each#20, Order

In [34]:
spark

In [35]:
df.unpersist()

DataFrame[Order ID: int, Product: string, Quantity Ordered: int, Price Each: double, Order Date: string, Purchase Address: string]

In [ ]:
broadcast всю таблицу на кждый воркер

In [37]:
spark.stop()